[View in Colaboratory](https://colab.research.google.com/github/calicinio/Testando/blob/master/Copy_of_IIC_TP_LSTMs.ipynb)

# Introdução à Inteligência Computacional
## Exercício sobre LSTM
## Autores:
* Álvaro Lemos
* Danilo Caldeira
* Felipe Carvalho
* Rogério Lima

## Algumas observações

### LSTM é complicado!
Pessoal, LSTM é uma técnica bastante complexa e de difícil entendimento, principalmente quando é a primeira vez que se estuda esse tema. Por conta disso, o texto introdutório está bem grande, pois ele tenta passar várias intuições relacionadas às LSTMs. É super tranquilo não entender todas as fórmulas que serão expostas aqui.


### Esse notebook pode parecer complicado... Mas não é :D
Tentamos mostrar através desse notebook a superioridade das LSTMs em relação às redes neurais tradicionais. Por conta disso, pode parecer que tem muito código e tudo mais, mas tentamos deixar tudo bem claro e, principalmente, a parte que vocês terão que responder é bastante tranquila. De todo modo, qualquer problema que vocês tiverem, não exitem em nos procurar. Nossos contatos são:

* alvarolemos@gmail.com
* felipefrecar93@gmail.com
* rlima.rogerio@gmail.com

Para quem tiver interesse em se aprofundar um pouco mais nesse tema, seguem algumas referências que achamos interessantes:

* [Blog post](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) *Understanding LSTMs*, do Christopher Olah. Trata-se de um texto bem didático e cheio de imagens para facilitar o entendimento
* [Curso online](https://www.coursera.org/learn/nlp-sequence-models) *Sequence Models*, ministrado pelo prof. Andrew Ng. Esse é o último curso da especialização de Deep Learning do Coursera. Trata-se de uma série de cursos extremamente didáticos e de mão na massa, recomendamos fortemente
* [Livro](http://www.deeplearningbook.org/) *Deep Learning*, um livro relativamente novo, cujos autores são personalidades muito conhecidas na comunidade de *Machine Learning*. O conteúdo aqui é muito mais aprofundado e não recomendamos que seja o primeiro contato


## O problema das sequências
As Redes Neurais *feed forward* (FFNN - *Feed Forward Neural Networks*) - as tradicionais apresentadas em sala - são boas para várias aplicações, mas pecam em muitos aspectos para problemas de sequência. Problemas de sequência são àqueles onde a ordem das características de uma amostra importa. Alguns exemplos são:

* Análise de sentimento
* Tradução de texto
* Previsão de séries temporais
* Speech-to-text
* etc.

São várias os problemas que uma FFNN encontra em um problema de sequência:

### Problema #1 - Sequências podem ter tamanho variável
FFNNs são treinadas para amostras com uma dimensão específica de entrada, mas sequências podem ter tamanhos variados. Por exemplo, um sistema de Análise de Sentimento para classificar críticas de filmes entre *positiva* e *negativa*, poderia ter que classificar amostras como:

* *"Filme excelente"* (tamanho = 2)
* *"Filme com atuação muito boa e excelente roteiro"* (tamanho = 8)
* *"Melhor filme desse ano"* (tamanho = 4)

Todas as críticas são claramente positivas, mas uma FFNN teria dificuldades em lidar elas, pois ela esperaria um número fixo de características no seu vetor de entradas

### Problema #2: Os padrões aprendidos pelos parâmetros não são compartilhados entre as posições das sequências
Isso fica evidente no problema anterior. Nas duas primeiras críticas, *filme* é a primeira palavra, enquanto que na terceira, ele é a segunda. Assim, os pesos relacionados direta (primeira camada escondida) ou indiretamente (nas mais profundas) a cada entrada precisariam aprendar a representação de todas as palavras, o que é bem ruim, pois nem todas as palavras aparecem em todas as posições. Seria necessário um conjunto de amostras muito maior para resolver esse problema. E mesmo que resolvesse, seriam necessários muitos neurônios e camadas para aprender o mesmo padrão individualmente, o que aumentaria o custo computacional consideravalmente.

Uma abordagem melhor para resolver esse tipo de problema é através de uma outra arquitetura de Redes Neurais, as Redes Recorrentes, que serão apresentadas a seguir.

## Redes Neurais Recorrentes
Diferetente das FFNN, as Redes Neurais Recorrentes (RNNs - *Recurrent Neural Networks*) lidam com sequências muito bem, pois foram feitas justamente para isso. O poder reside no fato de que cada amostra da sequência é computada individualmente por uma mesma rede, juntamente com o resultado das amostras anteriores, o que possibilita a representação não só da palavra, mas do contexto em que ela está.

Essa explicação ficará mais clara através da figura abaixo

![Diagrama de uma RNN](https://bitbucket.org/alvarolemos/lstm-notebook/raw/f1efe2926175e25f27f01454435214ec1d3e4a66/rnn_diagram.svg =300x)

Na figura acima, cada quadrado está realizando as seguintes operações

$$
a^{<t>} = g_1(W_a[a^{<t-1>},x^{<t>}] + b_a)
$$
$$
\hat{y}^{<t>} = g_2(W_{y}[a^{<t-1>},x^{<t>}] + b_y)
$$

onde $g_1$ é uma função de ativação $tanh$ ou $relu$ e $g_2$, uma $sigmoid$. Já $[a^{<t-1>},x^{<t>}]$ é uma notação para representar a concatenação entre $a^{<t-1>}$ e $x^{<t>}$. Ou seja, se $a^{<t-1>}$ tem dimensão $(3, 1)$ e $x^{<t>}$ $(4, 1)$, então $[a^{<t-1>},x^{<t>}]$ terá $(7, 1)$.

Na figura, o contexto das palavras anteriores é representada por $a^{<t>}$, que nada mais é que a ativação de uma camada de uma rede neural. Fica assim, claro que o contexto da frase vai sendo mantido ao longo da sequência. Outro ponto que é muito importante, é que pelas equações acima, vemos que para todas as palavras, são utilizados os mesmo pesos $W_{a}$, $W_{y}$, $b_a$ e $b_{y}$, o que resolve o **Problema #2** das FFNN. O **Problema #1** também é resolvido, uma vez que cada unidade recorrente (os quadradinhos!) são iguais, o que possibilita que frases de qualquer tamanho sejam aplicadas a um modelo RNN como o da figura.

## RNNs são bem legais mesmo... Mas então por que LSTMs?
Então, se RNNs fossem tão boas assim, esse notebook seria sobre elas e não sobre LSTM, certo? O ponto fraco das RNNs são sequências **muito longas**. Considere as seguintes frase:

*"**O gato comeu** maçã, laranja, banana e mais um tanto de coisa, então **ele está cheio** agora"*

*"**Os gatos comeram** maçã, laranja, banana e mais um tanto de coisa, então **eles estão cheios** agora"*

Talvez essa frase não seja tão longa assim, mas passa a ideia. Nessas ocasiões, as RNNs têm dificuldade de manter uma informação (plural, nesse caso) ativada ao longo de várias passos de uma sequência. Esse problema é conhecido como *vanishing gradients*, e de forma bem simplificada, o que ocorre é que, a medida que os a informação vai sendo passada a frente entre as células de $a^{<1>}$ para $a^{<2>}$, depois para $a^{<3>}$, e assim por diante, as ativações vão sendo multiplicadas sucessivamente pelos pesos $W_a$. Como os pesos em $W_a$ são normalmente valores dentro do intervalo $[0, 1]$, quão mais longa for uma sequência, menor será a ativação a cada passo. Assim, uma ativação do início da série (como *"O gato comeu"*) dificilmente alcança as células mais ao final da sequência. E mesmo que chegue, da mesma forma o backpropagation é afetado, ou seja, as atualizações dos pesos no início da sequência tendem a ser ínfimos.

Para lidar com esse problema, foi proposta a arquitetura de Redes Recorrentes LSTM (*"Long Short-Term Memory"*), que trouxe o conceito de memória de célula. A LSTM continua sendo recorrente, mas as operações que sua célula (quadrado da figura anterior) computa são diferentes. A figura abaixo mostra a nova estrutura. Ela é relativamente complexa, então vamos tentar entender a intuição por trás dela.

![Diagrama de uma RNN](https://github.com/Kulbear/deep-learning-coursera/raw/1beef8d17e804448c302b21a0b19833a7572f379/Sequence%20Models/images/LSTM.png =900x)

Vemos que, assim como numa RNN simples, a célula que processa os dados no instante $t$ recebe as ativações da célula anterior ($a^{<t-1>}$) e a entrada do instante atual ($x^{<t>}$), tendo como saída as ativações do instante atual. O que muda é que temos uma nova entrada, $c^{<t-1>}$, a memória da célula anterior, e uma nova saída, $c^{<t>}$, a memória da célula atual. Internamente, o que rege o funcionamento dessa *memória* são os portões de atualização e de esquecimento: $\Gamma_u$ (*update*) e $\Gamma_f$ (*forget*), respectivamente.

O que é importante nas suas fórmulas: ambos são resultados da aplicação de uma função sigmoidal ($\sigma(\cdot)$), ou seja, seus valores então entre $0$ e $1$. Por conta disso, o que é multiplicado por eles será reforçado ou atenuado, a depender se seus valores estiverem próximos de $1$ ou $0$, respectivamente.

Com isso em mente, repare que o portão de esquecimento ($\Gamma_f$) está sendo multiplicado por pelo estado da célula no instante anterior, $c^{<t-1>}$. Isso quer dizer que, se $\Gamma_f \approx 1$, o estado anterior será **reforçado**, ao passo que, se $\Gamma_f \approx 0$, ele será esquecido. Da mesma forma, o portão de atualização está sendo multiplicado pelo estado atual, ou seja, se  $\Gamma_u \approx 1$, a memória da célula será atualizada. E assim, o problema dos *vanishing gradients* está resolvido.

Vamos agora ao nosso exercício!

## Emojify
A aplicação que você irá completar aqui foi inspirada em um exercício do curso Sequence Models da especialização de Deep Learning do Coursera, em parceria com a deeplearning.ai. Essa aplicação funciona da seguinte forma: dada uma frase como entrada, ela retorna um emoji que transmita a mensagem que a frase de entrada está passando. Ela será implementada antes em uma FFNN, para vermos alguns problemas inerentes da incapacidade de lidar com sequência. Depois, em uma LSTM, onde poderemos ver sua superioridade para lidar com esse tipo de problema. 

Antes de começarmos, vale ressaltar uma nota sobre a representação das palavras de cada frase, que utiliza de **word embeddings**.

### Word Embeddings
A ideia por trás dos word embeddings é que eles possibilitam que palavras semelhantes tenham representações paracidas. Por exemplo, se durante o treinamento do Emojify, a frase *"hoje o almoço foi bom"* tinha como saída esperada o emoji 🍴 (talheres! :P), se o modelo aprendeu corretamente, muito provavelmente o mesmo emoji seria retornado para a frase *"hoje o jantar foi bom"*, pois ambos são substantivos que descrevem uma refeição.

Mas como ele sabe a representação das palavras *almoço* e *jantar*? Através de uma matriz de word embeddings, que já está pronta para ser usada. Existem algoritmos que treinam essas matrizes, mas esse problema não é o foco deste notebook. Existem vários word embeddings disponíveis online justamente para poupar o tempo de pessoas que querem implementar alguma aplicação de NLP (*Natural Language Processing* - Processamento de Linguagem Natural), sem precisar ter que treinar uma matriz de embeddings, que é uma tarefa muito cara computacionalmente.

Abaixo temos um exemplo de uma matriz de embeddings. As linhas são palavras do vocabulário do embedding, enquanto que as colunas são as dimensões de representação do mesmo. Nessa representação, os valores das colunas variam entre $[-1, 1]$ e quanto mais próximo desses limites estiver uma coluna de uma dada palavra, mais ela é representada por aquela característica. Vemos, por exemplo, que Homem e Mulher são muito bem representados pela característica gênero. Como são gêneros distintos, o sinal dessas palavras são opostos.

|            |   Gênero   |   Realeza  |    Idade   |  Alimento  |
| ---------- | ---------- | ---------- | ---------- | ---------- |
| Homem      |   -0.96    |   -0.02    |   -0.02    |    0.09    |
| Mulher     |   0.98     |   -0.03    |   -0.03    |    0.07    |
| Rei        |  -0.94     |  -0.93     |   0.09     |   0.00     |
| Rainha     |   0.92     |   0.98     |   0.12     |   0.01     |
| Maçã       |   0.00     |   0.01     |   0.01     |   0.98     |
| Laranja    |   0.01     |   0.03     |   0.04     |   0.97     |

É isso pessoal... Mãos a obra! ;)

# Exercício

Neste notebook resolveremos um problema de classificação de texto utilizando MLPs, que são modelos de redes neurais tradicionais e LSTMs que são modelos de redes neurais recorrentes (RNN).

Atualmente, a LSTM representa o estado da arte no que tange problemas de sequencias. LSTMs, por serem capazes de aprender hieraquias temporais tem comportamentos melhores para problemas com essas caracteristicas. Exemplos de problemas assim são problemas de classificação de texto por exemplo, onde a ordem das palavras pode mudar completamente o sentido da frase, de forma que palavras que aparecem antes em uma frase inflenciam quais aparecem depois além de influenciar outros fatores como conjulgação verbal, etc. 

Neste trabalho um dataset de frases onde cada label é um emoji correspondente à frase será utilizado e a performance de MLPs e LSTMs será comparada na tarefa de classificação. Ao final, deverá ficar evidente a capacidade superior da arquitetura LSTM de lidar melhor com modelos de sequencia.

In [1]:
#Baixando os dados necessários para o TP

!wget "https://docs.google.com/uc?export=download&id=1ECmCxGL2Sa36WtPVUc-qFWrCP14I4CmR" -O data.tar.gz
!tar -zxvf data.tar.gz
!pip install emoji

--2018-06-09 01:09:02--  https://docs.google.com/uc?export=download&id=1ECmCxGL2Sa36WtPVUc-qFWrCP14I4CmR
Resolving docs.google.com (docs.google.com)... 74.125.141.138, 74.125.141.113, 74.125.141.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.141.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-74-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qf5t1nb7ca4ulkprf3m4f4g8hj98kg6o/1528502400000/02256098521353866195/*/1ECmCxGL2Sa36WtPVUc-qFWrCP14I4CmR?e=download [following]
--2018-06-09 01:09:07--  https://doc-14-74-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qf5t1nb7ca4ulkprf3m4f4g8hj98kg6o/1528502400000/02256098521353866195/*/1ECmCxGL2Sa36WtPVUc-qFWrCP14I4CmR?e=download
Resolving doc-14-74-docs.googleusercontent.com (doc-14-74-docs.googleusercontent.com)... 74.125.141.132, 2607:f8b0:400c:c06::84
Connecting to doc-14-74-docs.googleusercontent.com (doc-14-74-d

In [2]:
import sys
sys.path.insert(0, './data')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import emoji
from utils import *

from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

%matplotlib inline

Using TensorFlow backend.


In [3]:
# Exemplo dos labels de treinamento e teste utilizados : Emojis
print_emoji_labels()

label: 0, emoji: ❤
label: 1, emoji: ⚾
label: 2, emoji: 😄
label: 3, emoji: 😞
label: 4, emoji: 🍴


## Data preprocessing

Os exemplos de treinamento e teste do dataset sao feitos de frases, e essas frases por consequencia constituidas por uma serie de palavras.

Uma representacao numerica a nivel de palavra (cada palavra sera posteriormente utilizada como entrada de um timestep da LSTM) será feita utilizado-se um embedding conhecido como glove vectors, de dimensão tamanho 50. Cade dimensao deste embedding pode ser intepretada como representando uma caracteristica da palavra, como sexo ou negacao/afirmacao, etc.

Embeddings, como ja discutido anteriormente sao uma forma muito poderosa de representar sequencias e texto, e foram um grande avanco na area de Deep Learning para esse tipo de modelo. Embeddings trazem uma capacidade de generalizacao muito boa para o problema, de forma que dados que nao necessariamente estao contidos no conjunto de treinamento, mas que estao no dicionario de embeddings do problema podem ser inferidos atraves do modelo por terem representacoes na forma de vetor parecidas.

Abaixo serão carregados embeddings pre treinados e criados dicionários que mapeam palavras para indices (sera usado posteriormente na LSTM), indices para palavras e palavras para sua representacao na forma de vetor glove 50 dimensional (os vetores serao utilizados em ambos modelos).


In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

In [5]:
# visualização dos conjuntos de treinamento
train_set = pd.read_csv('data/train_set.csv')
train_set.head()

,sentence,label
0,never talk to me again,3
1,I am proud of your achievements,2
2,It is the worst day in my life,3
3,Miss you so much,0
4,food is life,4


In [13]:
test_set = pd.read_csv('data/test_set.csv')
test_set.head()

,sentence,label
0,I want to eat,4
1,he did not answer,3
2,he got a very nice raise,2
3,she got me a nice present,2
4,ha ha ha it was so funny,2


In [14]:
print('Numero de amostras de treino: {}'.format(len(train_set)))
print('Numero de amostras de teste: {}'.format(len(test_set)))

Numero de amostras de treino: 132
Numero de amostras de teste: 56


Como pode ser visto o tamanho dos conjuntos de treinamento não são grandes, porem a quantidade de embeddings de palavras disponívels fazem com que mesmo com pouco dado o modelo ainda tenha uma capacidade de generalizacao para dados nao vistos relativamente alta, para os labels selecionados.

In [15]:
print('Numero de palavras no dicionario de embeddings: {}'.format(len(word_to_index)))

Numero de palavras no dicionario de embeddings: 400000


In [0]:
#turn into arrays
#train set
X_train = train_set['sentence'].values
y_train = train_set['label'].values
#test set
X_test = test_set['sentence'].values
y_test = test_set['label'].values

Como pode ser visto na visualizaçao dos dados acima, existe rotulos (labels) de 0 a 4 para os conjuntos de treinamento e teste.

Isso faz com que o nosso problema de classificao seja multi-classe.

Uma maneira de representar isto em redes neurais é fazer com que existam um numero de neuronios igual ao numero de classes que se deseja classificar. Dado uma ativação do tipo Softmax, o neuronio com maior valor de probabilidade atrelado recebe 1 e os demais 0.

Logo, y deve ser transformado em um vetor de 5 posicoes, onde deve ser colocado 1 naquela cujo indice corresponde ao label do y original. A esse tipo de representacao se da um nome de Onehot encoding.

Crie uma função que transforme um vetor y em um vetor one hot encoded (http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)



In [95]:
from sklearn.preprocessing import OneHotEncoder

one = OneHotEncoder()
new_y = np.reshape(y_test, (-1, 1))

type(y_test)
new_y.shape

#new_y = np.ndarray([len(y_test), 1])
#new_y = y_test

#one.fit(new_y)

#saida = one.transform(new_y)

#one.n_values_

#saida.toarray()


(56, 1)

In [0]:
from sklearn.preprocessing import OneHotEncoder

# Cria uma funcao que transform os Y em vetores one-hot. Basicamente, iremos de
# uma representacao do tipo:
#
#   3
#   2
#   1
#   4
# 
# Para algo do tipo:
#
#   0 0 1 0
#   0 1 0 0
#   1 0 0 0
#   0 0 0 1
#
# Para tal, utilize a classe OneHotEncoder do sklearn. O array de entrada deve ter
# uma coluna so, entao, para garantir isso, y.reshape(-1, 1). O resultado do one
# hot encoding deve ser transformado em em array, atravez da funcao .toarray()
def one_hot_encode(y):
    
    one = OneHotEncoder()
    y = np.reshape(y, (-1, 1))
    
    one.fit(y)
    
    y_1hot = one.transform(y).toarray()
    
    return y_1hot

y_train_1hot = one_hot_encode(y_train)
y_test_1hot = one_hot_encode(y_test)

**Expected output:**  
y_train_1hot.shape = (132, 5)  
y_test_1hot.shape = (56, 5)

In [98]:
print('y_train_1hot.shape = {}'.format(y_train_1hot.shape))
print('y_test_1hot.shape = {}'.format(y_test_1hot.shape))

y_train_1hot.shape = (132, 5)
y_test_1hot.shape = (56, 5)


Uma vez que o dataset foi preparado partiremos para o modelo de MLP seguido da LSTM. O input de cada um destes modelos e ligeiramente diferente.

No entanto vale ressaltar que, como ambos representam palavras na forma de vetores, e embora exista um dicionario de embeddings contendo 400k palavras, todas estas sao case sensitive e além disso cada exemplo de treinamento é constituido de uma frase. Logo, um pre processamento em comum para tanto a MLP quanto para a LSTM é garantir que todas as palavras tenham todas suas letras minusculas, assim como no dicionario, e alem disso que as frases sejam divididas em palavras individuais para cada observação.

## Modelo 1 - MLP

No caso da MLP, uma vez separadas cada palavra de uma frase, será tirado a media destas para servir como um exemplo de treinamento. Ou seja, supondo que a frase tenha 5 palavras por exemplo, a media destas 5 na sua forma de vetor 50 dimensional será um exemplo de treinamento para a rede MLP.

Isso se faz necessário pois as frases tem tamanho variável e o numero de caracteristicas na MLP nao pode ser variável, logo como solução tirou-se a media dos vetores de cada palavra. 

Vamos implementar a função que faz a média de uma frase utilizando os dicionarios de embeddings disponíveis:

In [99]:
#Exemplo
sentence = 'i LiKe poTaTos'
#use lower e split em uma frase para transformar tudo em minusculo e separar as palavras
words = sentence.lower().split()
print(words)

['i', 'like', 'potatos']


In [100]:
print(word_to_vec_map['potatos'])
print(word_to_vec_map['potatos'].shape)

[-0.77033  -0.57765  -1.4086    0.28488   1.3224    0.025708 -0.39524
  0.1565    0.73926   0.07319   0.39573   0.81557   1.7829   -1.1323
 -1.2555   -0.31784  -1.0604   -0.19973   2.2133   -0.13883   0.50197
  1.5084    0.58036   0.3317    0.56073   2.767     0.22387  -1.1203
  1.3909    0.51904   0.91258   1.2222    1.3356   -0.01816   1.855
  0.87767  -0.51949  -0.60569  -0.33336  -0.68257   0.18867  -1.2253
 -0.3839   -0.61717   1.311     1.0898    0.23086   1.3371   -0.060975
  1.4675  ]
(50,)


In [104]:
#média dessas 3 palavras em um unico vetor de 50 dimensoes, para ser um exemplo de treinamento para a MLP
avg = (word_to_vec_map[words[0]] + word_to_vec_map[words[1]] + word_to_vec_map[words[2]])/3

avg

array([-9.44466667e-02, -7.22533333e-02, -5.71287667e-01, -1.36759000e-01,
        7.60850000e-01, -6.08073333e-02, -4.92200000e-01, -3.22600000e-02,
       -1.98810000e-01,  1.93335857e-01,  4.30000000e-04,  7.90340000e-01,
        1.23240000e-01, -4.23786667e-01,  1.70286667e-01,  2.23070000e-01,
       -1.62426667e-01,  2.49016667e-01,  6.94771000e-01, -6.00296667e-01,
       -1.71700000e-01,  8.59043333e-01,  5.61106667e-01,  3.30116667e-01,
        6.74366667e-01, -5.22300000e-01, -7.96943333e-01, -1.19553333e-01,
        9.86853333e-01, -5.62653333e-01,  2.54052667e+00,  8.28670000e-01,
        2.11421000e-01,  8.96473333e-02,  5.04962000e-01,  2.39480000e-01,
       -1.62270000e-01,  6.08300000e-02,  1.46910000e-01, -3.42720000e-01,
       -3.12081000e-02, -1.48692667e-01, -2.21085733e-01,  1.30266667e-01,
        7.44175333e-01,  5.71426667e-01, -3.73400000e-02,  2.98733333e-02,
       -2.41591667e-01,  8.80180000e-01])

In [0]:
# implemente a funcao que dada uma frase, calcula a media da sua representacao na forma de vetor

def average_sentence(sentence):
    # a frase ja esta sendo lower e split pra voce :)
    sentence = sentence.lower().split()
    avg = np.zeros((50,))    
    n = len(sentence)
    for word in sentence:
        avg = word_to_vec_map[word] + avg
    avg = avg/n
    # FIM DO CODIGO
    return avg

**Expected output:**  

average_sentence('hello bob')

array([-0.729985  ,  0.49204   ,  0.196528  ,  0.17776   , -0.2934595 ,
        0.112665  , -1.169915  ,  0.05839   , -0.496285  , -0.1215    ,
       -0.4811865 ,  0.69023   , -0.03501   ,  0.2782145 ,  0.51474   ,
       -0.187681  , -0.124935  ,  0.4923305 , -0.114805  , -0.3723525 ,
       -0.29856   ,  0.616055  ,  0.331685  ,  0.00125   ,  0.82339   ,
       -0.874855  , -0.83978   ,  0.18487   ,  0.19692085, -0.681915  ,
        1.06786   ,  0.09266   , -0.401135  ,  0.4549565 , -0.65187   ,
       -0.22757   ,  0.39745   , -0.5354514 ,  0.2234785 , -0.585075  ,
        0.342345  ,  0.23009   , -0.89711   , -0.391322  ,  0.316475  ,
        0.071785  , -0.4854155 , -0.54965   , -0.3647985 ,  1.23727   ])

In [106]:
average_sentence('hello bob')

array([-0.729985  ,  0.49204   ,  0.196528  ,  0.17776   , -0.2934595 ,
        0.112665  , -1.169915  ,  0.05839   , -0.496285  , -0.1215    ,
       -0.4811865 ,  0.69023   , -0.03501   ,  0.2782145 ,  0.51474   ,
       -0.187681  , -0.124935  ,  0.4923305 , -0.114805  , -0.3723525 ,
       -0.29856   ,  0.616055  ,  0.331685  ,  0.00125   ,  0.82339   ,
       -0.874855  , -0.83978   ,  0.18487   ,  0.19692085, -0.681915  ,
        1.06786   ,  0.09266   , -0.401135  ,  0.4549565 , -0.65187   ,
       -0.22757   ,  0.39745   , -0.5354514 ,  0.2234785 , -0.585075  ,
        0.342345  ,  0.23009   , -0.89711   , -0.391322  ,  0.316475  ,
        0.071785  , -0.4854155 , -0.54965   , -0.3647985 ,  1.23727   ])

Utilizando a funcao anterior temos uma funcao que faz a mesma coisa para todos as observacoes de um dataset.

E aproveitando isso vamos calcular como ficaria o conjunto de teste e treino, uma vez tirado as medias das palavras de suas frases.

In [0]:
def average_dataset(X):
    m = X.shape[0]
    avg_X = np.zeros((m, 50))
    for i, sentence in enumerate(X):
        avg = average_sentence(sentence)
        avg_X[i,:] = avg
    return avg_X

avg_X_train = average_dataset(X_train)
avg_X_test = average_dataset(X_test)

In [108]:
#conferindo a dimensao para ver se temos um vetor com as 132 observacoes originais e 50 dimensoes
avg_X_train.shape

(132, 50)

Agora vamos finalmente construir um modelo MLP para classificar cada frase (que foi transformada numa media das palavras com 50 dimensoes) e ver seu emoji correspondente.

Vamos utilizar o framework Keras (https://keras.io/)

Primeiramente, para ficar familiarizado com o Keras, vamos construir um modelo de uma rede neural teste, uma MLP simples de 1 camada escondida com 5 neuronios e 1 camada de saida.


        

In [0]:
def modelo_teste(input_shape):
    #Vamos criar um placeholder para a entrada da rede
    #Input é uma funcao do Keras que faz isso
    #Este shape seria o numero de dimensoes do tensor
    #Por exemplo no nosso problema nosso vetor tem 50 dimensoes
    X_input = Input(input_shape)
    #Adicionando uma camada escondida com 5 neuronios
    #Dense significa fully connected: todos os pesos da camada anterior estao
    #conectados a todos os pesos da seguinte (padrao das MLPs)
    X = Dense(5)(X_input)
    #Repare como essa camada recebe a entrada X_input e retorna X
    #Esta camada tem uma ativacao, vamos escolher a sigmoid
    X = Activation('sigmoid')(X)
    #Agora uma camada de saida
    #Vamos supor um problema de classificacao com 3 classes para este teste (precisamos de 3 neuronios entao, seguindo nossa logica)
    X = Dense(3)(X)
    #uma ativacao sofmax é adequada
    X = Activation('softmax')(X)

    model = Model(inputs=X_input, outputs=X, name='MLP_teste')

Agora construa a MLP para nosso modelo de sequencia dos emojis.

A MLP tem uma camada escondida com 128 neuronios, ativacaco sigmoid, uma camada de saida com 5 neuronios, ativacao softmax.

In [0]:
def MLP_avg(input_shape):
    # SEU CODIGO AQUI
    X_input = Input(input_shape)
    X = Dense(128)(X_input)
    X = Activation('sigmoid')(X)
    X = Dense(5)(X)
    X = Activation('softmax')(X)
    # FIM DO CODIGO
    model = Model(inputs=X_input, outputs=X, name='MLP')
    
    return model   

  Vamos compilar e rodar pra ver os resultados dessa MLP das medias das palavras!
  
**Expected output:**  

  
Layer (type)                 Output Shape              Param #   

input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               6528      
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         

Total params: 7,173  
Trainable params: 7,173  
Non-trainable params: 0

In [113]:
MLP = MLP_avg((avg_X_train.shape[1],))
MLP.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
MLP.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               6528      
_________________________________________________________________
activation_3 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_4 (Activation)    (None, 5)                 0         
Total params: 7,173
Trainable params: 7,173
Non-trainable params: 0
_________________________________________________________________


In [115]:
MLP.fit(avg_X_train, y_train_1hot, epochs=400, batch_size=avg_X_train.shape[0], shuffle=True)

Epoch 1/400
132/132 [==============================] - 0s 38us/step - loss: 0.3298 - acc: 0.9318
Epoch 2/400
132/132 [==============================] - 0s 41us/step - loss: 0.3289 - acc: 0.9318
Epoch 3/400
132/132 [==============================] - 0s 30us/step - loss: 0.3281 - acc: 0.9318
Epoch 4/400
132/132 [==============================] - 0s 29us/step - loss: 0.3272 - acc: 0.9318
Epoch 5/400
132/132 [==============================] - 0s 18us/step - loss: 0.3263 - acc: 0.9318
Epoch 6/400
132/132 [==============================] - 0s 20us/step - loss: 0.3255 - acc: 0.9318
Epoch 7/400
132/132 [==============================] - 0s 26us/step - loss: 0.3246 - acc: 0.9318
Epoch 8/400
132/132 [==============================] - 0s 19us/step - loss: 0.3237 - acc: 0.9318
Epoch 9/400
132/132 [==============================] - 0s 22us/step - loss: 0.3229 - acc: 0.9318
Epoch 10/400
132/132 [==============================] - 0s 17us/step - loss: 0.3220 - acc: 0.9318
Epoch 11/400
132/132 [=======

Epoch 43/400
132/132 [==============================] - 0s 25us/step - loss: 0.2960 - acc: 0.9470
Epoch 44/400
132/132 [==============================] - 0s 25us/step - loss: 0.2953 - acc: 0.9470
Epoch 45/400
132/132 [==============================] - 0s 22us/step - loss: 0.2946 - acc: 0.9470
Epoch 46/400
132/132 [==============================] - 0s 23us/step - loss: 0.2938 - acc: 0.9470
Epoch 47/400
132/132 [==============================] - 0s 25us/step - loss: 0.2931 - acc: 0.9470
Epoch 48/400
132/132 [==============================] - 0s 26us/step - loss: 0.2924 - acc: 0.9470
Epoch 49/400
132/132 [==============================] - 0s 40us/step - loss: 0.2917 - acc: 0.9470
Epoch 50/400
132/132 [==============================] - 0s 27us/step - loss: 0.2910 - acc: 0.9470
Epoch 51/400
132/132 [==============================] - 0s 46us/step - loss: 0.2903 - acc: 0.9470
Epoch 52/400
132/132 [==============================] - 0s 28us/step - loss: 0.2895 - acc: 0.9545
Epoch 53/400
132/132

Epoch 77/400
132/132 [==============================] - 0s 30us/step - loss: 0.2728 - acc: 0.9545
Epoch 78/400
132/132 [==============================] - 0s 28us/step - loss: 0.2722 - acc: 0.9545
Epoch 79/400
132/132 [==============================] - 0s 32us/step - loss: 0.2715 - acc: 0.9545
Epoch 80/400
132/132 [==============================] - 0s 28us/step - loss: 0.2709 - acc: 0.9545
Epoch 81/400
132/132 [==============================] - 0s 41us/step - loss: 0.2703 - acc: 0.9545
Epoch 82/400
132/132 [==============================] - 0s 48us/step - loss: 0.2697 - acc: 0.9545
Epoch 83/400
132/132 [==============================] - 0s 20us/step - loss: 0.2690 - acc: 0.9545
Epoch 84/400
132/132 [==============================] - 0s 21us/step - loss: 0.2684 - acc: 0.9545
Epoch 85/400
132/132 [==============================] - 0s 34us/step - loss: 0.2678 - acc: 0.9545
Epoch 86/400
132/132 [==============================] - 0s 27us/step - loss: 0.2672 - acc: 0.9545
Epoch 87/400
132/132

Epoch 112/400
132/132 [==============================] - 0s 36us/step - loss: 0.2520 - acc: 0.9545
Epoch 113/400
132/132 [==============================] - 0s 22us/step - loss: 0.2514 - acc: 0.9545
Epoch 114/400
132/132 [==============================] - 0s 35us/step - loss: 0.2509 - acc: 0.9545
Epoch 115/400
132/132 [==============================] - 0s 28us/step - loss: 0.2503 - acc: 0.9545
Epoch 116/400
132/132 [==============================] - 0s 19us/step - loss: 0.2498 - acc: 0.9545
Epoch 117/400
132/132 [==============================] - 0s 19us/step - loss: 0.2492 - acc: 0.9545
Epoch 118/400
132/132 [==============================] - 0s 27us/step - loss: 0.2487 - acc: 0.9545
Epoch 119/400
132/132 [==============================] - 0s 29us/step - loss: 0.2482 - acc: 0.9545
Epoch 120/400
132/132 [==============================] - 0s 18us/step - loss: 0.2476 - acc: 0.9545
Epoch 121/400
132/132 [==============================] - 0s 20us/step - loss: 0.2471 - acc: 0.9545
Epoch 122/

132/132 [==============================] - 0s 36us/step - loss: 0.2352 - acc: 0.9621
Epoch 145/400
132/132 [==============================] - 0s 21us/step - loss: 0.2347 - acc: 0.9621
Epoch 146/400
132/132 [==============================] - 0s 27us/step - loss: 0.2342 - acc: 0.9621
Epoch 147/400
132/132 [==============================] - 0s 27us/step - loss: 0.2337 - acc: 0.9621
Epoch 148/400
132/132 [==============================] - 0s 29us/step - loss: 0.2332 - acc: 0.9621
Epoch 149/400
132/132 [==============================] - 0s 26us/step - loss: 0.2327 - acc: 0.9621
Epoch 150/400
132/132 [==============================] - 0s 27us/step - loss: 0.2322 - acc: 0.9621
Epoch 151/400
132/132 [==============================] - 0s 33us/step - loss: 0.2317 - acc: 0.9621
Epoch 152/400
132/132 [==============================] - 0s 46us/step - loss: 0.2313 - acc: 0.9621
Epoch 153/400
132/132 [==============================] - 0s 31us/step - loss: 0.2308 - acc: 0.9621
Epoch 154/400
132/132 [=

...


132/132 [==============================] - 0s 27us/step - loss: 0.1933 - acc: 0.9697
Epoch 243/400
132/132 [==============================] - 0s 21us/step - loss: 0.1930 - acc: 0.9697
Epoch 244/400
132/132 [==============================] - 0s 32us/step - loss: 0.1926 - acc: 0.9697
Epoch 245/400
132/132 [==============================] - 0s 26us/step - loss: 0.1922 - acc: 0.9697
Epoch 246/400
132/132 [==============================] - 0s 30us/step - loss: 0.1919 - acc: 0.9697
Epoch 247/400
132/132 [==============================] - 0s 29us/step - loss: 0.1915 - acc: 0.9697
Epoch 248/400
132/132 [==============================] - 0s 35us/step - loss: 0.1911 - acc: 0.9697
Epoch 249/400
132/132 [==============================] - 0s 28us/step - loss: 0.1908 - acc: 0.9697
Epoch 250/400
132/132 [==============================] - 0s 35us/step - loss: 0.1904 - acc: 0.9697
Epoch 251/400
132/132 [==============================] - 0s 27us/step - loss: 0.1901 - acc: 0.9697
Epoch 252/400
132/132 [

132/132 [==============================] - 0s 31us/step - loss: 0.1817 - acc: 0.9773
Epoch 276/400
132/132 [==============================] - 0s 19us/step - loss: 0.1814 - acc: 0.9773
Epoch 277/400
132/132 [==============================] - 0s 27us/step - loss: 0.1810 - acc: 0.9773
Epoch 278/400
132/132 [==============================] - 0s 30us/step - loss: 0.1807 - acc: 0.9773
Epoch 279/400
132/132 [==============================] - 0s 17us/step - loss: 0.1803 - acc: 0.9773
Epoch 280/400
132/132 [==============================] - 0s 43us/step - loss: 0.1800 - acc: 0.9773
Epoch 281/400
132/132 [==============================] - 0s 51us/step - loss: 0.1797 - acc: 0.9773
Epoch 282/400
132/132 [==============================] - 0s 43us/step - loss: 0.1793 - acc: 0.9773
Epoch 283/400
132/132 [==============================] - 0s 53us/step - loss: 0.1790 - acc: 0.9773
Epoch 284/400
132/132 [==============================] - 0s 48us/step - loss: 0.1787 - acc: 0.9773
Epoch 285/400
132/132 [=

132/132 [==============================] - 0s 45us/step - loss: 0.1719 - acc: 0.9773
Epoch 306/400
132/132 [==============================] - 0s 34us/step - loss: 0.1716 - acc: 0.9773
Epoch 307/400
132/132 [==============================] - 0s 32us/step - loss: 0.1713 - acc: 0.9773
Epoch 308/400
132/132 [==============================] - 0s 22us/step - loss: 0.1710 - acc: 0.9773
Epoch 309/400
132/132 [==============================] - 0s 25us/step - loss: 0.1707 - acc: 0.9773
Epoch 310/400
132/132 [==============================] - 0s 27us/step - loss: 0.1704 - acc: 0.9773
Epoch 311/400
132/132 [==============================] - 0s 20us/step - loss: 0.1701 - acc: 0.9773
Epoch 312/400
132/132 [==============================] - 0s 31us/step - loss: 0.1698 - acc: 0.9773
Epoch 313/400
132/132 [==============================] - 0s 38us/step - loss: 0.1695 - acc: 0.9773
Epoch 314/400
132/132 [==============================] - 0s 36us/step - loss: 0.1692 - acc: 0.9773
Epoch 315/400
132/132 [=

132/132 [==============================] - 0s 27us/step - loss: 0.1623 - acc: 0.9773
Epoch 338/400
132/132 [==============================] - 0s 23us/step - loss: 0.1620 - acc: 0.9773
Epoch 339/400
132/132 [==============================] - 0s 30us/step - loss: 0.1617 - acc: 0.9773
Epoch 340/400
132/132 [==============================] - 0s 27us/step - loss: 0.1614 - acc: 0.9773
Epoch 341/400
132/132 [==============================] - 0s 34us/step - loss: 0.1612 - acc: 0.9773
Epoch 342/400
132/132 [==============================] - 0s 36us/step - loss: 0.1609 - acc: 0.9773
Epoch 343/400
132/132 [==============================] - 0s 25us/step - loss: 0.1606 - acc: 0.9773
Epoch 344/400
132/132 [==============================] - 0s 36us/step - loss: 0.1603 - acc: 0.9848
Epoch 345/400
132/132 [==============================] - 0s 36us/step - loss: 0.1600 - acc: 0.9848
Epoch 346/400
132/132 [==============================] - 0s 26us/step - loss: 0.1597 - acc: 0.9848
Epoch 347/400
132/132 [=

Epoch 371/400
132/132 [==============================] - 0s 29us/step - loss: 0.1528 - acc: 0.9848
Epoch 372/400
132/132 [==============================] - 0s 43us/step - loss: 0.1526 - acc: 0.9848
Epoch 373/400
132/132 [==============================] - 0s 28us/step - loss: 0.1523 - acc: 0.9848
Epoch 374/400
132/132 [==============================] - 0s 33us/step - loss: 0.1520 - acc: 0.9848
Epoch 375/400
132/132 [==============================] - 0s 29us/step - loss: 0.1518 - acc: 0.9848
Epoch 376/400
132/132 [==============================] - 0s 31us/step - loss: 0.1515 - acc: 0.9848
Epoch 377/400
132/132 [==============================] - 0s 23us/step - loss: 0.1512 - acc: 0.9848
Epoch 378/400
132/132 [==============================] - 0s 32us/step - loss: 0.1510 - acc: 0.9848
Epoch 379/400
132/132 [==============================] - 0s 38us/step - loss: 0.1507 - acc: 0.9848
Epoch 380/400
132/132 [==============================] - 0s 24us/step - loss: 0.1504 - acc: 0.9848
Epoch 381/

Testando a acuracia no conjunto de teste:

In [116]:
loss, accuracy = MLP.evaluate(avg_X_test, y_test_1hot, batch_size=avg_X_test.shape[0])
print('Loss: {}, Acuracia: {}'.format(loss, accuracy))

56/56 [==============================] - 0s 593us/step
Loss: 0.285441726446, Acuracia: 0.875


Muito bom, a rede aprendeu bem nossos dados, vamos ver o que ela esta errando.

In [117]:
preds = MLP.predict(avg_X_test)
for i, phrase in enumerate(X_test):
    if np.argmax(preds[i]) != y_test[i]:
        x = X_test[i]
        y = y_test[i]
        y_emoji = label_to_emoji(y)
        y_hat = np.argmax(preds[i])
        y_hat_emoji = label_to_emoji(y_hat)
        print('Sentence: {}, Expected Label: {}, Predicted Label: {}'.format(x, y_emoji, y_hat_emoji))

Sentence: work is hard, Expected Label: 😞, Predicted Label: 😄
Sentence: This girl is messing with me, Expected Label: 😞, Predicted Label: ❤
Sentence: work is horrible, Expected Label: 😞, Predicted Label: 😄
Sentence: you brighten my day, Expected Label: 😄, Predicted Label: ❤
Sentence: she is a bully, Expected Label: 😞, Predicted Label: ❤
Sentence: My life is so boring, Expected Label: 😞, Predicted Label: ❤
Sentence: will you be my valentine, Expected Label: 😄, Predicted Label: ❤


Agora repare nesta frase e no output da rede:

In [118]:
def predict_emoji_MLP(model, sentence):
    pred = model.predict(average_sentence(sentence).reshape(1,-1))
    return label_to_emoji(np.argmax(pred))
  
print(predict_emoji_MLP(MLP, 'not feeling very happy'))

😄


In [119]:
print(predict_emoji_MLP(MLP, 'very happy not feeling'))

😄


Podemos ver que o 'not' nao esta sendo levado em conta, a MLP ve o happy e classifica emoji feliz.

A ordem da frase tambem pouco importa, nosso modelo ve afinal a media das palavras.

Um modelo de sequencia, que leve em conta a ordem temporal das palavras deve ser capaz de resolver estes problemas.

## Modelo 2 - LSTM

Para o modelo LSTM cada palavra das frases sera utilizada individualmente em vez de tirado a média das palavras como na MLP. Isso se deve ao fato de que cada palavra pode ocupar um timestep da LSTM e ser usada para tentar descobrir uma hierarquia temporal entre as palavras treinando o modelo de sequencia.

Por esse mesmo motivo, a LSTM nao tem problemas com frases de tamanhos variados, dado que cada observacao de treinamento pode ser treinada por um numero de timesteps equiavalente ao numero de palavras.

A saida da LSTM tambem sera uma camada Dense seguida de uma ativacao softmax, assim como na MLP.

Para fins de vetorizacao no Keras, embora em teoria a LSTM seja capaz de lidar com frases de tamanhos variados, sera feito um padding nas frases para que todas tenham um mesmo numero de palavras igual à maior frase. (por exemplo, se no conjunto de treino a maior frase tem 10 palavras, uma frase com 3 palavras tera 7 posicoes seguintes preenchidas com vetores de 0 uma vez que esta for transformada para o vetor de embeddings de 50 dimensoes). Vale ressaltar que isso so sera feito devido para ganhar desempenho na vetorizacao do treinamento com o keras.

In [0]:
#funcoes cujo objetivo e criar uma layer que mapea as palavras para seus embeddings no keras
#alem disso as frases sao 0 padded ate max_len (que no caso do nosso dataset e 10)

def sentence_to_index(X, word_to_index, max_len):
    m = X.shape[0]
    X_indices = np.zeros((m,max_len))
    for i in range(m):
        words = X[i].lower().split()[:max_len]
        for j, word in enumerate(words):
            X_indices[i, j] = word_to_index[word]
    return X_indices

def create_emb_matrix(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1
    emb_dim = 50 #glove vectors are 50D in our dataset
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    for word, index in word_to_index.items():
        emb_matrix[index,:] = word_to_vec_map[word]
        
    return emb_matrix

def create_emb_layer(emb_matrix):
    vocab_len, emb_dim = emb_matrix.shape
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

emb_matrix = create_emb_matrix(word_to_vec_map, word_to_index)
embedding_layer = create_emb_layer(emb_matrix)

Com a camada de embeddings pronta para traduzir as frases para seus vetores e coloca-los corretamente na LSTM, implemente o modelo da LSTM.

Este recebe 'embeddings' para uma camada LSTM de 128 neuronios (return_sequence=True, todos os outros parametros default), passa por uma tecnica de regularizacao chamada dropout com 0.5 de chance (nao se preocupe com isso, mas se quiser aprender sobre dropout saiba que e uma tecnica muito utilizada em deep learning, principalmente em visao computacional), depois uma outra camada LSTM de 128 neuronios (return_sequence=False, todos os outros parametros default), outro droput 0.5, uma camada densa de saida de 5 neuronios e uma ativacao softmax.

documentacao sobre LSTM : https://keras.io/layers/recurrent/

*oBS: entre a conecao das duas celulas de LSTM, ou seja, na primeira celula LSTM declarada, use return_sequences=True, que conecta cada timestep da primeira celula LSTM com a proxima.*

*ja na segunda celula lstm return_sequences=False, que significa que so no ultimo timestep da LSTM existe uma saida, que seria pra previsao de y*

![](https://github.com/Kulbear/deep-learning-coursera/raw/1beef8d17e804448c302b21a0b19833a7572f379/Sequence%20Models/images/emojifier-v2.png)

In [0]:
def LSTM_emoji(input_shape, embedding_layer):
    indices = Input(input_shape, dtype='int32')
    embeddings = embedding_layer(indices)
    # SEU CODIGO AQUI
    # Adiciona a primeira camada de LSTM aqui. Devera ter 128 neuronios,
    # recebe embeddings e retorna X. Utilize return_sequences=True
    X = LSTM(128, return_sequences=True)(embeddings)
    # FIM DO CODIGO 
    X = Dropout(0.5)(X)
    # SEU CODIGO AQUI
    # Adiciona a segunda camada de LSTM aqui. Devera ter 128 neuronios,
    # recebe e retorna X. Utilize return_sequences=False
    X = LSTM(128, return_sequences=False)(X)
    # FIM DO CODIGO 
    X = Dropout(0.5)(X)
    X = Dense(5)(X)
    X = Activation('softmax')(X)
    model = Model(inputs=indices, outputs=X)
    return model

maxLen = len(max(X_train, key=lambda string: len(string.split())).split())
LSTM = LSTM_emoji((maxLen,), embedding_layer)

**Expected output:**  

Layer (type)                 Output Shape              Param #   

input_2 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         

Total params: 20,223,927
Trainable params: 223,877
Non-trainable params: 20,000,050

In [124]:
LSTM.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 645       
__________

Nao se assuste com o numero de parametros, destes muitos sao os embeddings que agora sao um layer do keras para este modelo de LSTM. Mas eles sao nao treinaveis e ja estao pre treinados, (embora possam ser treinaveis) entao nao vai demorar muito pra treinar o modelo.

In [125]:
LSTM.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train_indices = sentence_to_index(X_train, word_to_index, maxLen)
X_test_indices = sentence_to_index(X_test, word_to_index, maxLen)
LSTM.fit(X_train_indices, y_train_1hot, epochs=100, batch_size=X_train_indices.shape[0], shuffle=True)

Epoch 1/100
132/132 [==============================] - 1s 10ms/step - loss: 1.6237 - acc: 0.1212
Epoch 2/100
132/132 [==============================] - 0s 668us/step - loss: 1.5803 - acc: 0.3030
Epoch 3/100
132/132 [==============================] - 0s 653us/step - loss: 1.5551 - acc: 0.3106
Epoch 4/100
132/132 [==============================] - 0s 695us/step - loss: 1.5310 - acc: 0.3333
Epoch 5/100
132/132 [==============================] - 0s 681us/step - loss: 1.5055 - acc: 0.3258
Epoch 6/100
132/132 [==============================] - 0s 679us/step - loss: 1.5098 - acc: 0.3106
Epoch 7/100
132/132 [==============================] - 0s 680us/step - loss: 1.4881 - acc: 0.3333
Epoch 8/100
132/132 [==============================] - 0s 694us/step - loss: 1.4503 - acc: 0.4015
Epoch 9/100
132/132 [==============================] - 0s 680us/step - loss: 1.4348 - acc: 0.4015
Epoch 10/100
132/132 [==============================] - 0s 699us/step - loss: 1.3988 - acc: 0.4015
Epoch 11/100
132/132

Epoch 32/100
132/132 [==============================] - 0s 703us/step - loss: 0.4878 - acc: 0.8409
Epoch 33/100
132/132 [==============================] - 0s 728us/step - loss: 0.4655 - acc: 0.8409
Epoch 34/100
132/132 [==============================] - 0s 737us/step - loss: 0.3812 - acc: 0.8561
Epoch 35/100
132/132 [==============================] - 0s 711us/step - loss: 0.3725 - acc: 0.8636
Epoch 36/100
132/132 [==============================] - 0s 681us/step - loss: 0.3073 - acc: 0.8788
Epoch 37/100
132/132 [==============================] - 0s 693us/step - loss: 0.3544 - acc: 0.8939
Epoch 38/100
132/132 [==============================] - 0s 703us/step - loss: 0.3243 - acc: 0.8864
Epoch 39/100
132/132 [==============================] - 0s 689us/step - loss: 0.2936 - acc: 0.8939
Epoch 40/100
132/132 [==============================] - 0s 685us/step - loss: 0.2727 - acc: 0.8939
Epoch 41/100
132/132 [==============================] - 0s 779us/step - loss: 0.2927 - acc: 0.9167
Epoch 42/1

132/132 [==============================] - 0s 673us/step - loss: 0.0452 - acc: 0.9848
Epoch 64/100
132/132 [==============================] - 0s 795us/step - loss: 0.1503 - acc: 0.9318
Epoch 65/100
132/132 [==============================] - 0s 703us/step - loss: 0.0825 - acc: 0.9697
Epoch 66/100
132/132 [==============================] - 0s 648us/step - loss: 0.0242 - acc: 1.0000
Epoch 67/100
132/132 [==============================] - 0s 761us/step - loss: 0.0661 - acc: 0.9848
Epoch 68/100
132/132 [==============================] - 0s 676us/step - loss: 0.0786 - acc: 0.9773
Epoch 69/100
132/132 [==============================] - 0s 699us/step - loss: 0.0544 - acc: 0.9848
Epoch 70/100
132/132 [==============================] - 0s 695us/step - loss: 0.0410 - acc: 0.9924
Epoch 71/100
132/132 [==============================] - 0s 737us/step - loss: 0.0520 - acc: 0.9848
Epoch 72/100
132/132 [==============================] - 0s 710us/step - loss: 0.0696 - acc: 0.9773
Epoch 73/100
132/132 [=

Epoch 94/100
132/132 [==============================] - 0s 709us/step - loss: 0.0068 - acc: 1.0000
Epoch 95/100
132/132 [==============================] - 0s 790us/step - loss: 0.0082 - acc: 1.0000
Epoch 96/100
132/132 [==============================] - 0s 766us/step - loss: 0.0088 - acc: 1.0000
Epoch 97/100
132/132 [==============================] - 0s 646us/step - loss: 0.0130 - acc: 0.9924
Epoch 98/100
132/132 [==============================] - 0s 681us/step - loss: 0.0064 - acc: 1.0000
Epoch 99/100
132/132 [==============================] - 0s 707us/step - loss: 0.0054 - acc: 1.0000
Epoch 100/100
132/132 [==============================] - 0s 735us/step - loss: 0.0059 - acc: 1.0000


In [126]:
loss, accuracy = LSTM.evaluate(X_test_indices, y_test_1hot)
print('Loss: {}, Acuracia: {}'.format(loss, accuracy))

56/56 [==============================] - 0s 4ms/step
Loss: 0.68515251364, Acuracia: 0.857142865658


A LSTM tem uma acuracia ligeiramente melhor que a MLP no treino e teste, mas os resultados sao similares. Vamos ver o que ela errou no conjunto de teste:

In [127]:
preds = LSTM.predict(X_test_indices)
for i, phrase in enumerate(X_test):
    if np.argmax(preds[i]) != y_test[i]:
        x = X_test[i]
        y = y_test[i]
        y_emoji = label_to_emoji(y)
        y_hat = np.argmax(preds[i])
        y_hat_emoji = label_to_emoji(y_hat)
        print('Sentence: {}, Expected Label: {}, Predicted Label: {}'.format(x, y_emoji, y_hat_emoji))

Sentence: she got me a nice present, Expected Label: 😄, Predicted Label: ❤
Sentence: work is hard, Expected Label: 😞, Predicted Label: 😄
Sentence: This girl is messing with me, Expected Label: 😞, Predicted Label: ❤
Sentence: any suggestions for dinner, Expected Label: 🍴, Predicted Label: 😄
Sentence: you brighten my day, Expected Label: 😄, Predicted Label: ❤
Sentence: she is a bully, Expected Label: 😞, Predicted Label: 😄
Sentence: will you be my valentine, Expected Label: 😄, Predicted Label: ❤
Sentence: go away, Expected Label: 😞, Predicted Label: ⚾


Qual a diferenca principal entao entre os modelos?

In [128]:
def predict_emoji_LSTM(model, sentence):
    pred = model.predict(sentence_to_index(np.array([sentence]), word_to_index, maxLen).reshape(1,-1))
    return label_to_emoji(np.argmax(pred))
  
print(predict_emoji_LSTM(LSTM, 'not feeling very happy'))

😄


In [129]:
print(predict_emoji_LSTM(LSTM, 'it is not good'))

😄


In [130]:
print(predict_emoji_LSTM(LSTM, 'it is good not'))

😄


## Conclusão

Como pode ser visto, a LSTM leva em consideracao a ordem da sequencia, como esperado, a tornando mais apropriada para este tipo de problema, mostrando uma capacidade maior de entender o contexto da frase e portanto se provando um modelo mais robusto.

Enquanto que a MLP aprendeu bem o dataset, ela 'overfitta' mais pois mudando um pouco o contexto ou a ordem ela nao consegue lidar. Ja a LSTM aprende as dependencias temporais e portanto faz previsoes fora do conjunto de treino bem melhores, aprendendo de verdade. 

In [131]:
#tente voce com uma frase qualquer
print(predict_emoji_LSTM(LSTM, 'i need a coffee it is 3 am help'))

🍴
